<a href="https://colab.research.google.com/github/shoboske/wine-quality-deep-learning/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install ucimlrepo scikit-learn pandas numpy matplotlib tensorflow

In [6]:
from ucimlrepo import fetch_ucirepo
import numpy as np
from sklearn.model_selection import train_test_split

# fetch dataset
wine_quality = fetch_ucirepo(id=186)
red_wine = wine_quality.data.original.query('color == "red"')
# data (as pandas dataframes)

X = red_wine.drop(columns=['quality', 'color'])
y = red_wine['quality']

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

scaler = StandardScaler()

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# Feature Extraction using SelectKBest and ANOVA F-value
selector = SelectKBest(f_classif, k=5) # Select top 5 features
X_train_selected = selector.fit_transform(X_train, y_train.values.ravel())
X_test_selected = selector.transform(X_test)

scaler.fit(X_train_selected)
X_train_selected = scaler.transform(X_train_selected)
X_test_selected = scaler.transform(X_test_selected)

# Create a simple neural network classifier
mlp = MLPClassifier(hidden_layer_sizes=(40,80,80,80,80,80,80), max_iter=1000, alpha=1e-4, activation='relu',
                    solver='sgd', batch_size=50, early_stopping=True, validation_fraction=0.2,
                    random_state=None,learning_rate_init=.1)
mlp_tanh = MLPClassifier(hidden_layer_sizes=(20,80,80,80,80,20), max_iter=1000, alpha=1e-4, activation='relu',
                    solver='adam', batch_size=50, early_stopping=True, validation_fraction=0.2,
                    random_state=None,learning_rate_init=.1)

model_keras = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=X_train_selected.shape[1:]),
  tf.keras.layers.Dense(20, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(40, activation='relu'),
  tf.keras.layers.Dropout(0.3),
  tf.keras.layers.Dense(20, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(11),
  tf.keras.layers.Softmax()
])

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model_keras.compile(optimizer='sgd', loss=loss_fn, metrics=['accuracy'])

# Train the model
mlp.fit(X_train_selected, y_train.values.ravel())
mlp_tanh.fit(X_train_selected, y_train.values.ravel())
model_keras.fit(X_train_selected, y_train.values.ravel(), epochs=100)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/nn.py:609: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2858 - loss: 2.3126
Epoch 2/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3697 - loss: 2.1735
Epoch 3/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4194 - loss: 2.0149
Epoch 4/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4029 - loss: 1.8895
Epoch 5/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4241 - loss: 1.7525
Epoch 6/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4193 - loss: 1.6419
Epoch 7/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4515 - loss: 1.5470
Epoch 8/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4554 - loss: 1.4923
Epoch 9/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4726 - loss: 1.4083
Epoch 10/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4552 - loss: 1.4058
Epoch 11/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5256 - loss: 1.3238 
Epoch 12/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5070 - l

In [17]:
# Evaluate the model (you can use various metrics like accuracy, precision, recall, etc.)
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import pandas as pd

# Make predictions on the test set
y_pred = mlp.predict(X_test_selected)

accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred, average='weighted')
precision = precision_score(y_test, y_pred, average='weighted', zero_division=0.0)
f1 = f1_score(y_test, y_pred, average='weighted')

y_pred_tanh = mlp_tanh.predict(X_test_selected)

accuracy_tanh = accuracy_score(y_test, y_pred_tanh)
recall_tanh = recall_score(y_test, y_pred_tanh, average='weighted')
precision_tanh = precision_score(y_test, y_pred_tanh, average='weighted', zero_division=0.0)
f1_tanh = f1_score(y_test, y_pred_tanh, average='weighted')

y_pred_keras = model_keras.predict(X_test_selected)
# Convert predictions to discrete class labels by selecting the index of the highest probability
y_pred_keras_classes = np.argmax(y_pred_keras, axis=1)  # Get class labels

accuracy_keras = accuracy_score(y_test, y_pred_keras_classes)
recall_keras = recall_score(y_test, y_pred_keras_classes, average='weighted')
precision_keras = precision_score(y_test, y_pred_keras_classes, average='weighted', zero_division=0.0)
f1_keras = f1_score(y_test, y_pred_keras_classes, average='weighted')

data = [
    ["MLP (100,200)", accuracy, recall, precision, f1],
    ["MLP (500,500,500,500,500)", accuracy_tanh, recall_tanh, precision_tanh, f1_tanh],
    ["Keras (100,200,200)", accuracy_keras, recall_keras, precision_keras, f1_keras]
]


headers = ["Model", "Accuracy", "Recall", "Precision", "F1 Score"]

print(pd.DataFrame(data, None, headers))

25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step  
                       Model  Accuracy   Recall  Precision  F1 Score
0              MLP (100,200)   0.59500  0.59500   0.582387  0.574569
1  MLP (500,500,500,500,500)   0.37750  0.37750   0.142506  0.206906
2        Keras (100,200,200)   0.56375  0.56375   0.535623  0.524735
